## Getting started with error mitigation with Mitiq on Braket 

In this notebook we will look at [Mitiq](https://mitiq.readthedocs.io/en/stable/), a python library for performing quantum error mitigation, in tandem with performance features from Amazon Braket for quantum computational applications. In subsequent notebooks, we show how to utilize Mitiq with Amazon Braket, specifically focusing on utilizing program sets to orchestrate job submissions, reducing costs and improving accuracies for quantum jobs.

### Why do we need error mitigation?

Realistic quantum systems couple to external environments. These interactions can result in information loss or distortion of the system of interest. In digital quantum computing, we can understand this simply as errors affecting our simulation (for instance, bit-flip or phase errors), and the strength of the interaction typically translates to the frequency or strength of noise. In principle, given certain error rates, we can use tools from quantum error correction to fully correct, these errors, but these are beyond the capabilities of systems today. 

We can instead try to mitigate errors or harness the noisy signal within a noisy quantum system, up to the decay time of our system. Quantum error mitigation (QEM, see [Review](https://arxiv.org/abs/2210.00921)) techniques attempts to do just this, similar to improving the signal-to-noise ratio of a noisy signal, generally at the cost of repeated measurements. We define quantum error mitigation, or just error mitigation, as tools or techniques which can reduce or *mitigate* the effect of quantum errors in a system, using approaches *without active feedback*. That is, extra copies of states can be used, and post processing channels can be applied to a system, but we do not allow for active feedback. This has helped to theoretically define the limits of quantum error mitigation (see [Takagi et al.](https://www.nature.com/articles/s41534-022-00618-z)).

### The cost of error mitigation

Error mitigation techniques have been shown ([Takagi et al.](https://www.nature.com/articles/s41534-022-00618-z), [Quek et al.](https://www.nature.com/articles/s41567-024-02536-7)) to have a fundamentally exponential scaling to obtain unbiased results. That is, to correct all error for arbitrary input states requires an exponential number of shots. This exponential requirement is present in the variance of the estimation, which naively scales as $e^{\lambda \times IF \times N}$, where $N$ is the number of relevant gates, $IF$ is some limiting gate infidelity, and $\lambda$ relates to the method. Probabilistic error cancellation for instance, is lower bounded by $\lambda = 4$, while zero-noise extrapolation can reach $\lambda = 2$, and post-selection, $\lambda = 1$ ([Aharanov et al.](https://arxiv.org/abs/2503.17243)). 

We include multiple references at the end for the interested reader. Importantly, this cost is reduced based on the quality of the method, the depth of the gates, and the limiting gate fidelities, generally a two qubit error rate. 

However, it is critical to know what is worth spending more time and executions on. Utilizing program sets is only half the battle - by batching we can generally reduce costs, seeing up to 100x decreases in cost, but we also want to be aware of where our variances are coming from, and that we have allocated our resources accordingly. This is a challenging task, and generally can vary depending on problems, circuits, devices, and mitigation techniques.




### Using Mitiq and Braket

Mitiq is an open-source Python toolkit for implementing error mitigation techniques, which covers a breadth of different methodologies and tools. 

The Mitiq library provides numerous error mitigation approaches as pre-implemented strategies. These include (but are not limited to):

- Readout error mitigation (`mitiq.rem`)
- Zero-noise extrapolation (`mitiq.zne`)
- Probabilistic Error Cancellation (`mitiq.pec`)
- Probabilistic Error Amplification (`mitiq.pea`)
- Pauli Twirling (`mitiq.pt`)
- Classical Shadows (`mitiq.shadows`)
- Quantum Subspace Expansions for Stabilizers (`mitiq.qse`)
- Clifford Data Regression (`mitiq.cdr`)
- Virtual Distillation (`mitiq.vd`)
- Digital Dynamical Decoupling (`mitiq.ddd`)

and more. See the [documentation](https://mitiq.readthedocs.io/en/stable/guide/guide.html) for a full list of supported methods. 

#### Patterns in Mitiq



Mitiq contains two methods for applying error mitigation generally. The first can be thought of as a simple function call in which one passes a Circuit and the QEM technique parameters into a function `mitiq.xyz.execute_with_xyz`. This uses an [`executor`](https://mitiq.readthedocs.io/en/stable/guide/executors.html) object, which is detailed below. 

```
from mitiq.zne import execute_with_zne
zne_result = execute_with_zne(circuit, executor, *args)
```
The second unpacks this procedure, separating the creation of circuits and their execution and reassembly:

```
from mitiq.zne import construct_circuits, combine_results
modified_circuits = construct_circuits(circuit)
raw_data = executor(modified_circuits)
zne_result = combine_results(raw_data)
```

in the `mitiq_braket_tools.py` file, we develop a `braket_mitiq_executor` function which can easily be used with the first pattern, and meets the requirements of a `mitiq.executor.Executor` object. The second we will explore more in a future notebook, and is develop in the parent `tools` folder, i.e. in the `tools/program_set_tools.py` file. 

Further details can be found in the [Mitiq guide](https://mitiq.readthedocs.io/en/stable/guide/guide.html). 

#### Using Verbatim circuits

For most of these approaches, we will want to treat our circuits using the [Verbatim box](https://docs.aws.amazon.com/braket/latest/developerguide/braket-openqasm-verbatim-compilation.html). This creates a more reliable thread between the submitted circuits and the noise structure which is being treated, and also allows us to skip the Braket service compiler.

That is, given a circuit of interest, we may: 

1. Pass our circuit through a compiler / transpiler to nativize it
2. Pass our circuit through Mitiq to create copies or multiple instances
3. Run our circuits on the Braket service using Verbatim compilation 

For simplicity, we will use the [Qiskit-Braket provider](https://github.com/qiskit-community/qiskit-braket-provider/tree/main) to harness the Qiskit transpiler, though alternatively the Braket compiler can be used as well by submitting a job to the service against a compiler and returning the output circuit. 


### Installing Mitiq

> Mitiq is not included by default with the amazon-braket-examples or in the Amazon Braket notebook instances. To install, uncomment the first line in the next code block, and if necessary, restart the notebook. Mitiq utilizes [Cirq](https://quantumai.google/cirq) as the backend, and often times will represent circuits using their representation. 

In [1]:
# %pip install mitiq cirq-ionq
try:
    import mitiq  # noqa: F401
    print("Package 'mitiq' is installed.")
except ImportError:
    print("Package 'mitiq' is not installed.")

Package 'mitiq' is installed.


We can then import other packages. Throughout we use pre-packaged noise models or emulators, which are detailed in the noise_model.py file, and can all be carried out locally. In the final notebook we look at a QPU-related example, which has associated costs. 

In [2]:
import os
import sys

from mitiq.observable import Observable, PauliString

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))   

from qiskit_braket_provider import to_braket, to_qiskit
from tools.noise_models import qd_depol

from braket.circuits import Circuit

In [3]:
## UNCOMMENT_TO_RUN
# from braket.devices import LocalSimulator
# qd_depol = LocalSimulator()

### Executors and Observables

[Executors](https://mitiq.readthedocs.io/en/stable/guide/executors.html) are the main engines for running mitiq. These are quite flexible, and are detailed more on the mitiq page.

Generally these can be user-defined functions which input Circuits and output either floats or Measurement-type results. The `braket_expectation_executor` and `braket_measurement_executor` are Executors for Braket [Observables](https://amazon-braket-sdk-python.readthedocs.io/en/latest/_apidoc/braket.circuits.observables.html) or mitiq [Observables](https://mitiq.readthedocs.io/en/stable/guide/observables.html), respectively. 

> The Executors are essentially wrappers around ProgramSets, and will fail it you try to submit more executables than the program set limit of a given device. 

In [4]:
from mitiq_braket_tools import braket_expectation_executor, braket_measurement_executor

Below we perform a simple zero-noise extrapolation experiment using the `braket_expectation_executor` executor, i.e. the Executor receives a float output.  

In [ ]:
import warnings

from mitiq.zne import RichardsonFactory, execute_with_zne

from braket.circuits.observables import Z

base_circ = Circuit().h(0).cnot(0,1)
base_circ = to_braket(to_qiskit(base_circ, add_measurements=False), basis_gates=["rz","rx", "cx"])
print(base_circ)

# initialize executor 
executor = braket_expectation_executor(qd_depol, Z(0) @ Z(1), shots = 10000, verbatim= False, batch_if_possible=False)

# noisy result as a reference 
noisy_result = executor.evaluate(base_circ)[0]
print(f"Noisy result: {noisy_result}")

# perform the ZNE in one step 

factory = RichardsonFactory([1., 3., 5.])
zne_result = execute_with_zne(base_circ, executor, factory=factory)
print(f"Zero-noise extrapolation Result: {zne_result}")

# now, test average performance 
noisy, zne = [], []
for i in range(10):
    warnings.filterwarnings("ignore", category=UserWarning)
    noisy.append(executor.evaluate(base_circ)[0])
    zne.append(execute_with_zne(base_circ, executor, factory=factory))

avg_noisy = sum([abs(1-a) for a in noisy])/10
avg_zne = sum([abs(1-a) for a in zne])/10

print(f"{100*(1 - (avg_zne)/(avg_noisy)):.2f}% average reduction in error")

/Users/sesmart/coding/development/braket-examples/local/amazon-braket-examples/.venv/lib/python3.12/site-packages/qiskit_braket_provider/providers/adapter.py:1025: UserWarning: Device does not support global phase; global phase of 1.5707963267948966 will not be included in Braket circuit
  warnings.warn(


T  : │     0      │     1      │     2      │  3  │
      ┌──────────┐ ┌──────────┐ ┌──────────┐       
q0 : ─┤ Rz(1.57) ├─┤ Rx(1.57) ├─┤ Rz(1.57) ├───●───
      └──────────┘ └──────────┘ └──────────┘   │   
                                             ┌─┴─┐ 
q1 : ────────────────────────────────────────┤ X ├─
                                             └───┘ 
T  : │     0      │     1      │     2      │  3  │
Noisy result: 0.9544
Zero-noise extrapolation Result: 0.9969999999999999


/Users/sesmart/coding/development/braket-examples/local/amazon-braket-examples/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(


88.48% average reduction in error


In comparison, the `braket_measurement_executor` returns a `mitiq.MeasurementResult`, and can support mitiq-based Observables in the executor. 

In [6]:
zz = Observable(PauliString("ZZ", coeff = 1)) #mitiq Observable and PauliString

executor_2 = braket_measurement_executor(qd_depol, 10000, False, batch_if_possible=False)

noisy_result = executor_2.evaluate(base_circ, zz)[0]
print(f"Noisy Result: {noisy_result}")

factory = RichardsonFactory([1., 3., 5.])
zne_result = execute_with_zne(base_circ, executor_2, zz,  factory=factory)
print(f"Zero-noise extrapolation Result: {zne_result}")


Noisy Result: (0.9526+0j)
Zero-noise extrapolation Result: (1.0132249999999998+0j)


## Calibrators

We can also use Mitiq's built in [`Calibrator`](https://mitiq.readthedocs.io/en/stable/guide/calibrators.html), to assess which approaches may be most suitable for a particular Executor, method, or Circuit. These can be defined for certain protocols or backends. The `Calibrator` below focuses on variations of ZNE. The total cost can be easily calculated as $0.3 + 100* C_{shot} * N_{spe}$, where $N_{spe}$ is the number of shots per executable, and $C_{shots}$ is the cost per shot given for a particular device. We can already see that using program sets, we can submit a single calibration for $0.30$, as opposed to 100 evaluations, i.e. $30. 

More details are given on the Calibrator [here](https://mitiq.readthedocs.io/en/stable/examples/calibration-tutorial.html). 

In [7]:
from mitiq.calibration import Calibrator

cal = Calibrator(executor_2, frontend = "braket")

print(cal.get_cost())


{'noisy_executions': 100, 'ideal_executions': 0}


In [8]:
cal.run()

Below we can see how the Calibrator performs for variants of ZNE on different types of reference circuits and two-qubit gate counts. 

In [9]:
cal.results.log_results_cartesian()

┌────────────────────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────┐
│ strategy\benchmark                 │ Type: ghz                  │ Type: w                    │ Type: rb                   │ Type: mirror               │
│                                    │ Num qubits: 2              │ Num qubits: 2              │ Num qubits: 2              │ Num qubits: 2              │
│                                    │ Circuit depth: 2           │ Circuit depth: 2           │ Circuit depth: 49          │ Circuit depth: 33          │
│                                    │ Two qubit gate count: 1    │ Two qubit gate count: 2    │ Two qubit gate count: 13   │ Two qubit gate count: 14   │
├────────────────────────────────────┼────────────────────────────┼────────────────────────────┼────────────────────────────┼────────────────────────────┤
│ Technique: ZNE                     │ ✘                          │ ✔ 

Here we can see that mitiq will perform a scan of multiple methods at a lower accuracy, and provide recommendations for which approach achieves the best. With a particular gate structure, folding strategy, etc., and noise rate, this can provide a hands-off approach for assessing viable approaches.

We can also inspect our Executor, looking at relevant outputs and the total number of circuits run. 

In [10]:
executor_2.calls_to_executor

104

### Summary

In this notebook we saw an overview of mitiq, as well as how one method to utilize mitiq with program sets. Generally, error mitigation allows us to reliably expand what is possible on today's quantum computers, and can help deliver more reliable and useful results for noisy quantum devices. 

In subsequent notebooks we will dive into common error mitigation strategies, and provide explicit implementations which you can utilize for systems of interest.

### References
1. Unitary Foundation, *Mitiq Documentation*, https://mitiq.readthedocs.io/en/stable/index.html. Accessed 12/1/2025. 
2. Cai et al., [*Quantum error mitigation*](https://arxiv.org/abs/2210.00921) (2023) Rev. Mod. Phys., 95, 045005. 
3. Wang et al., [*Amazon Braket introduces program sets enabling customers to run quantum programs up to 24x faster*](https://aws.amazon.com/blogs/quantum-computing/amazon-braket-introduces-program-sets-enabling-customers-to-run-quantum-programs-up-to-24x-faster/), AWS Quantum Technologies Blog. Accessed 12/1/2025.
4. Quek et al., [*Exponentially tighter bounds on limitations of quantum error mitigation*](https://www.nature.com/articles/s41567-024-02536-7) (2024), Nat Phys. 20, 1648-1658. 
5. Takagi et al., [*Fundamental limits of quantum error mitigation*](https://www.nature.com/articles/s41534-022-00618-z) (2022)  npj Quantum Inf. 8, 114. 
6. Aharonov et al. [*On the importance of Error Mitigation for Quantum Computation*](https://arxiv.org/abs/2503.17243) (2025), arxiv:2503.17243.